# FHIR dbt Analytics Example Exploration

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google/medical_claims_tools/blob/main/analytics/bigquery_bcda_exporation.ipynb)

This notebook contains basic examples of exploring FHIR dbt data
in BigQuery. The results saved below are from the metrics from synthetic example data, but this notebook can be run against any  data exported from FHIR Store to BigQuery.



## Setup


In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

from google.cloud import bigquery
bqclient = bigquery.Client(project="dqi-playground-teams")


Authenticated


In [3]:
#@title Parameters
bigquery_table_prefix = "dqi-playground-teams.dbt_synthea_fhirstore" #@param {type:"string"}


## Metrics summary

In [20]:
query_str = """
SELECT 
  primary_resource,
  description, 
  IF (numerator IS NOT NULL, numerator / denominator, denominator) measure,
  denominator
FROM (
  SELECT 
    primary_resource,
    description, 
    SUM(numerator) numerator,
    SUM(denominator_cohort) denominator,
    
  FROM `{bigquery_table_prefix}.metric_by_slices_system_date`
  GROUP BY 1, 2
  ORDER BY 1, 2
  LIMIT 20
);
""".format(bigquery_table_prefix=bigquery_table_prefix)
bqclient.query(query_str).result().to_dataframe()
# dataframe.plot.bar(rot=0, x="eob_code", title="Claims by EOB Type", ylabel="Count", xlabel="EOB Type Code",figsize=(8,5))

,primary_resource,description,measure,denominator
0,AllergyIntolerance,Count of valid AllergyIntolerance resources,0.000000,0
1,CareTeam,Count of valid CareTeam resources,12.000000,12
2,Condition,Count of valid Condition resources,112.000000,112
3,DiagnosticReport,Count of valid DiagnosticReport resources,348.000000,348
4,DiagnosticReport,Proportion of diagnostic reports with body sit...,0.000000,348
5,Encounter,Count of valid Encounter resources,203.000000,203
6,Encounter,Proportion of Encounter resources that do not ...,0.000000,203
7,MedicationAdministration,Count of valid MedicationAdministration resources,5.000000,5
8,MedicationRequest,Count of valid MedicationRequest resources,127.000000,127
9,Observation,Count of valid Observation resources,989.000000,989
